In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.models import load_model
from keras.layers import Dropout,Conv1D, MaxPooling1D, Flatten,TimeDistributed,Bidirectional
import math
import datetime as dt
from datetime import datetime    
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.layers import MaxPooling1D, Flatten
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import RootMeanSquaredError


def import_data_from_csv(path: str):
    """
    Imports data from a csv file and returns a pandas dataframe.
    """
    return pd.read_csv(path)

def reshape_data(df):
    X = []
    Y = []
    window_size=100
    for i in range(1 , len(df) - window_size -1 , 1):
        first = df.iloc[i,4]
        temp = []
        temp2 = []
        for j in range(window_size):
            temp.append((df.iloc[i + j, 4] - first) / first)
        temp2.append((df.iloc[i + window_size, 4] - first) / first)
        X.append(np.array(temp).reshape(100, 1))
        Y.append(np.array(temp2).reshape(1, 1))
    return X,Y

def get_train_and_test_values(df):
    X,Y = reshape_data(df)
    train_X_values,test_X_values,train_Y_values,test_Y_values = [],[],[],[]
    
    num_splits = 5
    split_size = len(X) / num_splits
    
    X_splits = [X[int(i * split_size):int((i + 1) * split_size)] for i in range(num_splits)]
    Y_splits = [Y[int(i * split_size):int((i + 1) * split_size)] for i in range(num_splits)]
    
        
    for i in range(num_splits):
        x_train, x_test, y_train, y_test = train_test_split(X_splits[i], Y_splits[i], test_size=0.2, shuffle=True)
        train_X = np.array(x_train)
        test_X = np.array(x_test)
        train_Y = np.array(y_train)
        test_Y = np.array(y_test)
        
        train_X = train_X.reshape(train_X.shape[0],1,100,1)
        test_X = test_X.reshape(test_X.shape[0],1,100,1)
        
        train_X_values.append(train_X)
        test_X_values.append(test_X)
        train_Y_values.append(train_Y)
        test_Y_values.append(test_Y)

    return train_X_values,test_X_values,train_Y_values,test_Y_values

def create_model():
    model = tf.keras.Sequential()
    
    # Creating the Neural Network model here...
    # CNN layers
    model.add(TimeDistributed(Conv1D(64, kernel_size=3, activation='relu', input_shape=(None, 100, 1))))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Conv1D(128, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Conv1D(64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Flatten()))
    
    # LSTM layers
    model.add(Bidirectional(LSTM(100, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(100, return_sequences=False)))
    model.add(Dropout(0.5))
    
    #Final layers
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

    return model


def train_model(company_name,model,train_X_values,test_X_values,train_Y_values,test_Y_values):
    for i in range(len(train_X_values)):
        model.fit(train_X_values[i], train_Y_values[i], validation_data=(test_X_values[i],test_Y_values[i]), epochs=20,batch_size=40, verbose=1, shuffle =True)
        model.save("algorithms/cnn-lstm/checkpoints/"+company_name+"/cnn-lstm-"+str(i+1)+".keras")
    return model

def get_test_X_and_test_Y_data(data2):
    data2.dropna(inplace=True)
    data2.reset_index(drop=True, inplace=True)
    df2 = data2.drop('Date', axis=1)
    
    X,Y = reshape_data(df2)
    
    test_X = np.array(X)
    test_Y = np.array(Y)
    
    test_X = test_X.reshape(test_X.shape[0],1,100,1)

    return test_X,test_Y

def evaluate_model(model,test_X,test_Y):
    model.evaluate(test_X, test_Y)
    predicted  = model.predict(test_X)
    test_label = test_Y.reshape(-1,1)
    predicted = np.array(predicted[:,0]).reshape(-1,1)
    len_t = len(test_X)
    print(len_t)
    for j in range(len_t):
        temp = data2.iloc[j,3]
        test_label[j - len_t] = test_label[j - len_t] * temp + temp
        predicted[j - len_t] = predicted[j - len_t] * temp + temp
    plt.plot(test_label, color = 'red', label = 'Real Stock Price')
    plt.plot(predicted, color = 'green', label = 'Predicted  Stock Price')
    plt.title(' Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel(' Stock Price')
    plt.legend()
    plt.show()

def train_model_for_stocks(company_name):
    file_name = "data\\processed\\"
    file_name += company_name+'.csv'
    data = import_data_from_csv(file_name)
    train_X_values,test_X_values,train_Y_values,test_Y_values = get_train_and_test_values(data)
    model = create_model()
    model = train_model(company_name,model,train_X_values,test_X_values,train_Y_values,test_Y_values)
    return model
    

def main():
    train_model_for_stocks("AAPL")

if __name__ == "__main__":
    main()

Epoch 1/20



13/13 [==============================] - 20s 289ms/step - loss: 0.0350 - mse: 0.0350 - mae: 0.1471 - val_loss: 0.0187 - val_mse: 0.0187 - val_mae: 0.1065
Epoch 2/20
13/13 [==============================] - 0s 37ms/step - loss: 0.0144 - mse: 0.0144 - mae: 0.0928 - val_loss: 0.0105 - val_mse: 0.0105 - val_mae: 0.0824
Epoch 3/20
13/13 [==============================] - 0s 31ms/step - loss: 0.0091 - mse: 0.0091 - mae: 0.0739 - val_loss: 0.0066 - val_mse: 0.0066 - val_mae: 0.0641
Epoch 4/20
13/13 [==============================] - 0s 33ms/step - loss: 0.0067 - mse: 0.0067 - mae: 0.0648 - val_loss: 0.0057 - val_mse: 0.0057 - val_mae: 0.0615
Epoch 5/20
13/13 [==============================] - 0s 33ms/step - loss: 0.0059 - mse: 0.0059 - mae: 0.0622 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 0.0619
Epoch 6/20
13/13 [==============================] - 0s 33ms/step - loss: 0.0052 - mse: 0.0052 - mae: 0.0586 - val_loss: 0.0046 - val_mse: 0.0046 - val_mae: 0.0550
Epoch 7/20
13/13 

FileNotFoundError: [Errno 2] No such file or directory: 'algorithms/cnn-lstm/checkpoints/AAPL/cnn-lstm-1.keras'